<a href="https://colab.research.google.com/github/MCanela-1954/DataSci_Course/blob/main/%5BDATA-02E%5D%20Example%20-%20Airbnb%20Barcelona%20listings%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [DATA-02E] Example - Airbnb Barcelona listings data

## Introduction

**Airbnb** is a peer-to-peer online marketplace and homestay network, which enables people to list or rent short-term lodging in residential properties, with the cost of accommodation set by the property owner, called the **host** at Airbnb. The company receives percentage service fees from both guests and hosts in conjunction with every booking. Starting in 2008, it has grown exponentially, and it currently has over 6 million listings in about 200 countries.

Airbnb currently releases and updates data at the **Inside Airbnb** site (`www.insideairbnb.com`). The updates posted in March 2023 cover 116 areas, most of them in US and Europe. This example uses data from Barcelona, of a total of on 15,655 **listings**. In Barcelona, vacation apartments are subject to the highest rate of property tax, and platforms must share data with regulators.

## The data set

The data come in the file `airbnb.csv`. The columns of the source file are:

* `listing_id`, a unique listing's ID. An active listing is a property listed on Airbnb. Listings may include entire homes or apartments, private rooms or shared spaces.

* `host_id`, a unique host's ID.

* `host_since`, the date of the host's first listing in Airbnb, as 'yyyy-mm-dd'.

* `name`, the listing's name. A minimal description (maximum 35 characters) of the place, intended to be appealing, such as 'Centric Bohemian next Ramblas & Macba'. The language is typically, but not always, either English or Spanish.

* `neighbourhood`, the **neighbourhood** (barrio) of the listing. The neighbourhoods are sourced from the city.

* `district`, the district of the listing. The districts, called **neighbourhood groups** at Airbnb, are sourced from the city. There are 10 districts in Barcelona, each containing several neighbourhoods.

* `property_type`, the type of property listed. 75% of the listings come as 'Entire rental unit', or 'Private room in rental unit', but Airbnb admits many other possibilities.

* `room_type`, taking values 'Entire home/apt', 'Private room', 'Shared room' and 'Hotel room'.

* `bedrooms`, the number of available bedrooms.

* `price`, the daily listing's price on that date, in euros. The price shown is for the listing as a whole, not per person. The price that you see when you search Airbnb for specific dates is the total price divided by the number of nights spcified. When a listing has been booked for several days, the price can be lower, since the host can apply different prices depending on the number of days booked.

* `number_of_reviews`, the number of reviews of that listing that have been posted.

* `review_scores_rating`, the average reviewers' rating of overall experience (*What was your guest’s overall experience?*). Listings are rated in the range 1-100.

Source: `http://insideairbnb.com/get-the-data`.

## Questions

Q1. How many **duplicates** do you find in this data set?

Q2. What is the proportion of listings whose rating is **missing**?

Q3. Use a histogram to explore the distribution of the price.

Q4. What is the average price per room type?

Q5. In which neighbourhoods do we find more listings? Are they more expensive?

## Importing the data

We import Pandas as usual.

In [ ]:
import pandas as pd

The source files for this course can be extracted from a GitHub repository. The **path** is:

In [ ]:
path = 'https://raw.githubusercontent.com/MCanela-1954/Data/main/'

To import the data to a Pandas data frame, we use the function `read_csv()`.

In [ ]:
df = pd.read_csv(path + 'airbnb.csv')

## Exploring the data

The report printed by the method `.info()` contains basic information about the data. We have 15,655 data points, as expected. Among the 11 columns, `name`, `bedrooms` and `review_scores_rating` have missing values, in different proportions.

In [ ]:
df.info()

The first rows can be displayed with the method `.head()`.

In [ ]:
df.head()

## Q1. How many duplicates do you find in this data set?

Duplicates can be detected with the method `.duplicated()`, which returns a Boolean series, with value `True` if all the entries in the corresponding row is duplicated, and `False` otherwise. This method uses a top-down search.

These Booleans are stored in the series returned by `.duplicated()`. By applying `.sum to this Boolean series, we convert the Booleans to integers (1/0), so the sum is equal to the number of `True` values, that is, the number of duplicates. This calculation illustrates how computers count the times an expression is true. Note that, if the same row appears $n$ times in the data set, it is counted as $n - 1$ duplicates.

In this example, it makes sense to apply this method to the first column, to check whether there are duplicated listing ID's. So, we start here.

In [ ]:
df['listing_id'].duplicated().sum()

So, the listing ID is a good ID. Nevertheless, when we check whether the data of the listings are duplicated, we find 28 cases. They correspond to listings that have exactly the same data. This may happen, *e.g*. when they refer to beds in a shared bedroom.

In [ ]:
df.drop(columns='listing_id').duplicated().sum()

## Q2. Proportion of listings with missing ratings

The report extracted with `.info()` already let us know that some columns have missing values. In particular, we have a relevant number of listings for which rating scores are not available (many guests don't rate the lodgings). A specific report on missingness, which can be restricted to some selected columns, can be extracted with the method `.isna()`, which returns a Boolean data frame of the same shape indicating whether an entry is missing.

By applying `.sum()` after `.isna()`, we would obtain the column totals, that is, the numbers of missing values for every column. Or, by applying `.mean()`, we obtain the number of missing values (*i.e*. the column totals) divided by the number of listings (*i.e*. the number of rows). So, we get the proportion of missing values for every column as:

In [ ]:
df.isna().mean().round(3)

We find a 22.1% of listings with no review scores.

## Q3. Distribution of the price

A histogram for the prices can be obtained with the method `.plot.hist()`, which has already been used in this course.

In [ ]:
df['price'].plot.hist(title='Figure 1. Distribution of the price',
    color='gray', edgecolor='white', xlabel='Price per night (euros)');

Is this histogram useful? Not much, since some very expensive lodgings distort the whole picture, which is a frequent issue with skewed distributions. These extreme prices look like errors and, indeed, by using the listing's ID, it can be checked that some of them don't match the prices given in the Airbnb website. Another perspective of the distribution can be provided by the statistical summary printed by the method `.describe()`.

In [ ]:
df['price'].describe()

For a better picture of the bulk of Airbnb listings we could **trim the data**. For instance, we can plot a histogram for the listings with prices within a reasonable interval. To get such histogram, we can use the **filter expression** such as:

In [ ]:
filter_expr = df['price'].between(25,175)

The role of this expression is obvious. An equivalent expression would be `(df['price'] >= 25) & (df['price'] <= 175)`. Note that the method `.between()` (copied from an SQL function) includes the two limits of the interval.

Next, we draw a histogram including the listings that pass the filter.

In [ ]:
df['price'][filter_expr].plot.hist(title='Figure 2. Distribution of the price (trimmed data)',
    color='gray', edgecolor='white', bins=30, xlabel='Price per night (euros)');

The argument `bins=30` ensures that the range of the data is partitioned into intervals whose limits are multiples of 5, which makes the figure more appealing. the histogram illustrates a well known fact, that prices don't change continuously, and hosts prefer prices that are multiples of 10. The histogram also shows that 50, 100 and 150 euros are popular prices.

*Note*. The default for counting the observations in every bin is: for an interval of limits $a$ and $b$, the values $x$ such that $a \le x < b$ are counted. Except for the last bin, for which the right limit is also counted.


## Q4. Average price per room type

A **pivot table** with the average price per room type can be calculated with the method `.pivot_table()`. The parameters `values` and `index` specify the column to be aggregated and the column to be used for grouping, respectively.

In [ ]:
table1 = pd.pivot_table(df, values='price', index='room_type', aggfunc='mean').round()
table1

But, how informative is the average price? With skewed distributions, the extreme observations on the right tail "pull" the mean, so it may fall far from the middle of the distribution. In these cases, the **median** gives a better description of the mid prices. To get the median price per room type, we use `aggfunc='median'` (no need of rounding here).

In [ ]:
table2 = pd.pivot_table(df, values='price', index='room_type', aggfunc='median')
table2

You may prefer to display these tables as **bar charts**. Note that, even if we see a pivot table just as something that the Python kernel prints to the screen, these methods (except `.info()`) return new Pandas objects. Our second pivot table, for instance, is a Pandas series, with the room types as the index. So, we can display a bar chart with the method `.plot.bar()`.

In [ ]:
table2.plot.bar(title='Figure 3. Median price per room type', xlabel='Room type',
    legend=False, color='gray', ylabel='Price per night (euros)');

## Q5. Top-10 neighbourhoods

We take now a look at the neighbourhoods with more listings. The top ten list can be extracted with the method `.value_counts()`, which returns a series with counts of the occurrences of the values of a series, sorted top down. These neighbourhood names come as the index.

In [ ]:
table3 = df['neighbourhood'].value_counts().head(10)
table3

Are these neighbourhoods more expensive? We wonder if there is an association between the mid price and the amount of property listed. To get the answer, we could use a pivot table with the neighbourhood in the rows and the number of listings and the mid price in the columns. Instead, we apply here the method `.groupby()` (copied from SQL), with two aggregate functions. The function `count()` counts the non missing observations.

Except for la Dreta de l'Eixample, we don't find higher prices associated to more listings. So, the picture is more complex than that.

In [ ]:
table4 = df.groupby(by='neighbourhood')['price'].agg(['count', 'median']).sort_values(by='count',
    ascending=False).head(10)
table4

## Mounting the dirve

Pandas provides a toolkit to export/import tabular data sets, which is quite simple. Just as the function `read_csv()` and the method `.to_csv()` allow us to switch from Pandas data frames to CSV files and conversely, similar tools exist for JSON and SQL.

Except when a path is explicitly specified, files are read from and written to the **working directory**. The current working directory is then a folder called `content`, whose contents you can see in the browser by clicking in the *Files* button on the left side on the screen. Currently, we have there a single folder called `sample_data`.

Since anything that we store in `content` is deleted when the current session is closed, it is preferrable to store our data files in the MyDrive folder of your drive, with the rest of our files. To have access from the Python kernel to this folder, we have to **mount** first the drive, which is done as follows. Google will ask you for identification and consent.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

We can now use any path starting with `'/content/drive/'`. For instance:

In [ ]:
mypath = '/content/drive/MyDrive/'

## Exporting to a JSON file

We can now export our data to a JSON file, that will appear in the folder specified in the path we use. Since a JSON object can have various structures, combining square and curly braces in different ways, we have to control this with the parameter `orient`. The argument `orient='records'` will create a data structure that, in Python, will be seen as a list of dictionaries, each dictionary accounting for a row of the data frame. You can check that this is so in the browser.

In [ ]:
df.to_json(mypath + 'airbnb.json', orient='records')

The data can be imported back to a Pandas data frame. Let us use a different name, `df_json`.

In [ ]:
df_json = pd.read_json(mypath + 'airbnb.json')

Let us check that everything is right.

In [ ]:
df_json.info()

# Exporting to an SQL database


Though relational databases share the SQL language (not completely standardized), they are different, so we need a specific **driver** to connect to a database from a external application (not only from Python). In Python, those drivers come, as many other things, as packages. For this example, we are using **SQLite**, which is serverless, so a database is just a special file that you can save in computer or in your cloud space.

SQLite3 comes with any Python distribution, and the package `sqlite3` is part of the Python Standard Library. We import this directly.

In [ ]:
import sqlite3

To work with a database, you must connect to that database. To connect to an SQLite database from Python, you can do it as follows (this is not the only way).

In [ ]:
conn = sqlite3.connect(mypath + 'iese.db')

This creates the database with the specified path (if it didn't exist), but the corresponding file will not show up in Google Drive until we close the connection. We export our data set to a table called `airbnb` with the method `.to_sql()`. The argument `index=False` stops the index to be included as the first column in that table.

In [ ]:
df.to_sql('airbnb', conn, index=False)

We are done, so can close the connection.

In [ ]:
conn.close()

In [ ]:
ls

You can see the files `airbnb.json` and `iese.db` in MyDrive (you may need to refresh).

# SQL queries from Pandas

We can run queries to our database from the Python kernel. First, we reconnect.

In [ ]:
conn = sqlite3.connect(mypath + 'iese.db')

Let us respond to questions Q4 and Q5 with a couple of easy queries. For the first table that we extracted to respond question Q4, the query would be:

In [ ]:
q4 = 'SELECT AVG(price), room_type FROM airbnb GROUP BY room_type;'

In [ ]:
q4_output = pd.read_sql(q4, conn)
q4_output

Medians are not available in SQLite, so we leave this as it is. For the last table of question Q5, we would do (again with means instead of medians):

In [ ]:
q5 = '''
    SELECT neighbourhood, COUNT(*) AS count, ROUND(AVG(price), 1) AS price
    FROM airbnb
    GROUP BY neighbourhood
    ORDER BY count DESC
    LIMIT 10
'''
q5_output = pd.read_sql(q5, conn)
q5_output

In [ ]:
conn.close()